https://github.com/Jays-code-collection/HMMs_Stock_Market

first im going to decompose his code into a notebook, and then combine it and explain it in my own way

the hidden markov model is used to score the likelihood of an event occuring given the previous 10 days

can i use the one day predictions to do one day expiring options?

In [1]:
import warnings
import logging
import itertools
import pandas as pd
import numpy as np
from pandas_datareader import data
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import argparse
import sys
import os
from datetime import timedelta
#gjhgjhgjhgjhgjhgjhg

warnings.filterwarnings("ignore")

Setup variables

In [106]:
company = "AAPL"
start_date = "2018-01-01"
end_date = "2020-12-05"
n_latency_days = 10
hmm = GaussianHMM(n_components=4) # 4 is number of hidden states
predicted_close = None # not sure what for
days_in_future = 5

n_intervals_frac_change=50
n_intervals_frac_high=10
n_intervals_frac_low=10
test_size=0.33

In [107]:
# download the data (split train test)

import yfinance as yfin
yfin.pdr_override()

used_data = data.get_data_yahoo(company, start_date, end_date)

train_data, test_data = train_test_split(
            used_data, test_size=test_size, shuffle=False
        )

train_data = train_data.drop(["Volume", "Adj Close"], axis=1)
test_data = test_data.drop(["Volume", "Adj Close"], axis=1)

days = len(test_data)

[*********************100%%**********************]  1 of 1 completed


In [108]:
# Extract the test features of the data
def _extract_features(data):
    """Extract the features - open, close, high, low price - from the Yahooo finance generated dataframe."""
    open_price = np.array(data["Open"])
    close_price = np.array(data["Close"])
    high_price = np.array(data["High"])
    low_price = np.array(data["Low"])

    # We compute the fractional change in high,low and close prices to use as our set of observations
    frac_change = (close_price - open_price) / open_price
    frac_high = (high_price - open_price) / open_price
    frac_low = (open_price - low_price) / open_price

    # Put the observations into one array
    return np.column_stack((frac_change, frac_high, frac_low))

Fit the hidden markov model to the train data

In [109]:
observations = _extract_features(train_data)
hmm.fit(observations)

GaussianHMM(n_components=4)

compute all possible outcomes

Creates np arrays with evenly  spaced numbers for each range.

In [110]:
frac_change_range = np.linspace(-0.1, 0.1, n_intervals_frac_change)
frac_high_range = np.linspace(0, 0.1, n_intervals_frac_high)
frac_low_range = np.linspace(0, 0.1, n_intervals_frac_low)

_possible_outcomes = np.array(
    list(itertools.product(frac_change_range, frac_high_range, frac_low_range))
)


Using the fitted HMM, calculate the most probable outcome for a given day (e.g. prices will rise by 0.01).
        :param day_index: Current day index
        :return: The HMM's predicted movements in frac_change, frac_high, frac_low

In [117]:
day_index = 2

# Use the previous n_latency_days worth of data for predictions
previous_data_start_index = max(0, day_index - n_latency_days)
previous_data_end_index = max(0, day_index - 1)
previous_data = test_data.iloc[
    previous_data_start_index:previous_data_end_index
]
previous_data_features = _extract_features(previous_data)

outcome_score = []

# Score all possible outcomes and select the most probable one to use for prediction
for possible_outcome in _possible_outcomes:
    total_data = np.row_stack((previous_data_features, possible_outcome))
    outcome_score.append(hmm.score(total_data))

# Get the index of the most probable outcome and return it
most_probable_outcome = _possible_outcomes[np.argmax(outcome_score)]

most_probable_outcome

array([-0.00204082,  0.01111111,  0.01111111])

Predict close price for a given day. (using most probable outcome)

In [19]:
open_price = test_data.iloc[day_index]["Open"]
(
    predicted_frac_change,
    pred_frac_high,
    pred_frac_low,
) = most_probable_outcome
predicted_close_price = open_price * (1 + predicted_frac_change)

predicted_close_price

75.11297867444097

Predict close prices for the testing period.


:return: List object of predicted close prices

In [60]:
class HMMStockPredictor:
    def __init__(
        self,
        company,
        start_date,
        end_date,
        future_days,
        test_size=0.33,
        n_hidden_states=4,
        n_latency_days=10,
        n_intervals_frac_change=50,
        n_intervals_frac_high=10,
        n_intervals_frac_low=10,
    ):
        self._init_logger()
        self.company = company
        self.start_date = start_date
        self.end_date = end_date
        self.n_latency_days = n_latency_days
        self.hmm = GaussianHMM(n_components=n_hidden_states)
        self._split_train_test_data(test_size)
        self._compute_all_possible_outcomes(
            n_intervals_frac_change, n_intervals_frac_high, n_intervals_frac_low
        )
        self.predicted_close = None
        self.days_in_future = future_days

    def _init_logger(self):
        self._logger = logging.getLogger(__name__)
        handler = logging.StreamHandler()
        formatter = logging.Formatter(
            "%(asctime)s %(name)-12s %(levelname)-8s %(message)s"
        )
        handler.setFormatter(formatter)
        self._logger.addHandler(handler)
        self._logger.setLevel(logging.DEBUG)

    def _split_train_test_data(self, test_size):
        """Downloads data and splits it into training and testing datasets."""
        # Use pandas_reader.data.DataReader to load the required financial data. Checks if the stock entry is valid.

        #* fixing error
        import yfinance as yfin
        yfin.pdr_override()

        try:
            # used_data = data.DataReader(
            #     self.company, "yahoo", self.start_date, self.end_date
            # )
            used_data = data.get_data_yahoo(self.company, self.start_date, self.end_date)
        except IOError:
            print(
                "Invalid stock selection. Please try again with a stock that is available on Yahoo finance."
            )
            sys.exit()
        # Do not shuffle the data as it is a time series
        _train_data, test_data = train_test_split(
            used_data, test_size=test_size, shuffle=False
        )
        self.train_data = _train_data
        self.test_data = test_data

        # Drop the columns that aren't used
        self.train_data = self.train_data.drop(["Volume", "Adj Close"], axis=1)
        self.test_data = self.test_data.drop(["Volume", "Adj Close"], axis=1)

        # Set days attribute
        self.days = len(test_data)

    @staticmethod
    def _extract_features(data):
        """Extract the features - open, close, high, low price - from the Yahooo finance generated dataframe."""
        open_price = np.array(data["Open"])
        close_price = np.array(data["Close"])
        high_price = np.array(data["High"])
        low_price = np.array(data["Low"])

        # We compute the fractional change in high,low and close prices to use as our set of observations
        frac_change = (close_price - open_price) / open_price
        frac_high = (high_price - open_price) / open_price
        frac_low = (open_price - low_price) / open_price

        # Put the observations into one array
        return np.column_stack((frac_change, frac_high, frac_low))

    def fit(self):
        """Fit the continuous emission Gaussian HMM."""
        self._logger.info(">>> Extracting Features")
        observations = HMMStockPredictor._extract_features(self.train_data)
        self._logger.info("Features extraction Completed <<<")
        # Fit the HMM using the fit feature of hmmlearn
        self.hmm.fit(observations)

    def _compute_all_possible_outcomes(
        self, n_intervals_frac_change, n_intervals_frac_high, n_intervals_frac_low
    ):
        """Creates np arrays with evenly  spaced numbers for each range."""
        frac_change_range = np.linspace(-0.1, 0.1, n_intervals_frac_change)
        frac_high_range = np.linspace(0, 0.1, n_intervals_frac_high)
        frac_low_range = np.linspace(0, 0.1, n_intervals_frac_low)

        self._possible_outcomes = np.array(
            list(itertools.product(frac_change_range, frac_high_range, frac_low_range))
        )

    def _get_most_probable_outcome(self, day_index):
        """
        Using the fitted HMM, calculate the most probable outcome for a given day (e.g. prices will rise by 0.01).
        :param day_index: Current day index
        :return: The HMM's predicted movements in frac_change, frac_high, frac_low
        """
        # Use the previous n_latency_days worth of data for predictions
        previous_data_start_index = max(0, day_index - self.n_latency_days)
        previous_data_end_index = max(0, day_index - 1)
        previous_data = self.test_data.iloc[
            previous_data_start_index:previous_data_end_index
        ]
        previous_data_features = HMMStockPredictor._extract_features(previous_data)

        outcome_score = []

        # Score all possible outcomes and select the most probable one to use for prediction
        for possible_outcome in self._possible_outcomes:
            total_data = np.row_stack((previous_data_features, possible_outcome))
            outcome_score.append(self.hmm.score(total_data)) # get the log likelihood of that outcome occuring

        # Get the index of the most probable outcome and return it
        most_probable_outcome = self._possible_outcomes[np.argmax(outcome_score)]

        return most_probable_outcome

    def predict_close_price(self, day_index):
        """Predict close price for a given day."""
        open_price = self.test_data.iloc[day_index]["Open"]
        (
            predicted_frac_change,
            pred_frac_high,
            pred_frac_low,
        ) = self._get_most_probable_outcome(day_index)
        return open_price * (1 + predicted_frac_change)

    def predict_close_prices_for_period(self):
        """
        Predict close prices for the testing period.
        :return: List object of predicted close prices
        """
        predicted_close_prices = []
        print(
            "Predicting Close prices from "
            + str(self.test_data.index[0])
            + " to "
            + str(self.test_data.index[-1])
        )
        for day_index in tqdm(range(self.days)):
            predicted_close_prices.append(self.predict_close_price(day_index))
        self.predicted_close = predicted_close_prices
        return predicted_close_prices

    def real_close_prices(self):
        """ "Store and return the actual close prices."""
        actual_close_prices = self.test_data.loc[:, ["Close"]]
        return actual_close_prices

    def add_future_days(self):
        """
        Add rows to the test data dataframe for the future days being predicted with accurate days. The rows are left
        with NaN values for now as they will be populated whilst predicting.
        """
        last_day = self.test_data.index[-1] + timedelta(days=self.days_in_future)

        # Create a new df with future days x days in the future based off the -f input. Concat the new df with
        # self.test_data.
        future_dates = pd.date_range(
            self.test_data.index[-1] + pd.offsets.DateOffset(1), last_day
        )
        second_df = pd.DataFrame(
            index=future_dates, columns=["Open", "High", "Low", "Close"]
        )
        self.test_data = pd.concat([self.test_data, second_df])

        # Replace the opening price for the first day in the future with the close price of the previous day
        self.test_data.iloc[self.days]["Open"] = self.test_data.iloc[self.days - 1][
            "Close"
        ]

    def predict_close_price_fut_days(self, day_index):
        """
        Predict the close prices for the days in the future beyond the available data and populate the DF accordingly.
        :param day_index - index in DF for  current day being predicted.
        :return: Predicted close price for given day.
        """
        open_price = self.test_data.iloc[day_index]["Open"]

        # Calculate the most likely fractional changes using the trained HMM
        (
            predicted_frac_change,
            pred_frac_high,
            pred_frac_low,
        ) = self._get_most_probable_outcome(day_index)
        predicted_close_price = open_price * (1 + predicted_frac_change)

        # Fill in the dataframe based on predictions
        self.test_data.iloc[day_index]["Close"] = predicted_close_price
        self.test_data.iloc[day_index]["High"] = open_price * (1 + pred_frac_high)
        self.test_data.iloc[day_index]["Low"] = open_price * (1 - pred_frac_low)
        print(predicted_close_price)

        return predicted_close_price

    def predict_close_prices_for_future(self):
        """
        Calls the "predict_close_price_fut_days" function for each day in the future to predict future close prices.
        """
        predicted_close_prices = []
        future_indices = len(self.test_data) - self.days_in_future
        print(
            "Predicting future Close prices from "
            + str(self.test_data.index[future_indices])
            + " to "
            + str(self.test_data.index[-1])
        )
        self.test_data.iloc[future_indices]["Open"] = self.test_data.iloc[
                    future_indices - 1
                ]["Close"]
        # Iterate over only the final x days in the test data dataframe.
        for _day_index in tqdm(range(future_indices, len(self.test_data))):
            predicted_close_prices.append(self.predict_close_price_fut_days(_day_index))
            # Replace the next days Opening price (which is currently NaN) with the previous days predicted close price
            try:
                self.test_data.iloc[_day_index + 1]["Open"] = self.test_data.iloc[
                    _day_index
                ]["Close"]
            except IndexError:
                continue

        # Return the predicted close prices
        self.predicted_close = predicted_close_prices

        return predicted_close_prices

In [122]:
company = 'AAPL'
start_date = '2020-01-01'
end_date = '2023-01-01'
days_in_future = None
stock_predictor = HMMStockPredictor(company=company, start_date=start_date, end_date=end_date, future_days=days_in_future)
stock_predictor.fit()
# future
# stock_predictor.add_future_days()

# future_pred_close = stock_predictor.predict_close_prices_for_future()

[*********************100%%**********************]  1 of 1 completed

2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,372 __main__     INFO     >>> Extracting Features
2023-09-23 19:37:37,

create a strategy where if the open price of the day is predicted to go up past some threshold, buy the stock by some weight, if it is thought to go down past a threshold then sell by some weight. Exit all trades at the end of each day

maybe retrain the model on the past months data?

In [123]:
company = "^GSPC"
start_date = "2020-01-01"
end_date = "2023-01-01"
n_latency_days = 10
hmm = GaussianHMM(n_components=4) # 4 is number of hidden states
predicted_close = None # not sure what for
days_in_future = 5

n_intervals_frac_change=50
n_intervals_frac_high=10
n_intervals_frac_low=10
test_size=0.33

In [124]:
# download the data (split train test)

import yfinance as yfin
yfin.pdr_override()

used_data = data.get_data_yahoo(company, start_date, end_date)

train_data, test_data = train_test_split(
            used_data, test_size=test_size, shuffle=False
        )

train_data = train_data.drop(["Volume", "Adj Close"], axis=1)
test_data = test_data.drop(["Volume", "Adj Close"], axis=1)

days = len(test_data)

[*********************100%%**********************]  1 of 1 completed


In [125]:
# Extract the test features of the data
def _extract_features(data):
    """Extract the features - open, close, high, low price - from the Yahooo finance generated dataframe."""
    open_price = np.array(data["Open"])
    close_price = np.array(data["Close"])
    high_price = np.array(data["High"])
    low_price = np.array(data["Low"])

    # We compute the fractional change in high,low and close prices to use as our set of observations
    frac_change = (close_price - open_price) / open_price
    frac_high = (high_price - open_price) / open_price
    frac_low = (open_price - low_price) / open_price

    # Put the observations into one array
    return np.column_stack((frac_change, frac_high, frac_low))

In [126]:
observations = _extract_features(train_data)
hmm.fit(observations)

Model is not converging.  Current: 5287.529451104843 is not greater than 5289.702126282505. Delta is -2.1726751776614037


GaussianHMM(n_components=4)

In [127]:
frac_change_range = np.linspace(-0.1, 0.1, n_intervals_frac_change)
frac_high_range = np.linspace(0, 0.1, n_intervals_frac_high)
frac_low_range = np.linspace(0, 0.1, n_intervals_frac_low)

_possible_outcomes = np.array(
    list(itertools.product(frac_change_range, frac_high_range, frac_low_range))
)
_possible_outcomes

array([[-0.1       ,  0.        ,  0.        ],
       [-0.1       ,  0.        ,  0.01111111],
       [-0.1       ,  0.        ,  0.02222222],
       ...,
       [ 0.1       ,  0.1       ,  0.07777778],
       [ 0.1       ,  0.1       ,  0.08888889],
       [ 0.1       ,  0.1       ,  0.1       ]])

In [139]:

def get_most_probable_outcome(_day_index):
    # Use the previous n_latency_days worth of data for predictions
    previous_data_start_index = max(0, _day_index - n_latency_days)
    previous_data_end_index = max(0, _day_index - 1)
    previous_data = test_data.iloc[
        previous_data_start_index:previous_data_end_index
    ]
    previous_data_features = _extract_features(previous_data)

    outcome_score = []
    # Score all possible outcomes and select the most probable one to use for prediction
    for possible_outcome in _possible_outcomes:
        total_data = np.row_stack((previous_data_features, possible_outcome))
        outcome_score.append(hmm.score(total_data))

    # Get the index of the most probable outcome and return it
    most_probable_outcome = _possible_outcomes[np.argmax(outcome_score)]

    return most_probable_outcome
get_most_probable_outcome(19)

array([-0.00204082,  0.        ,  0.01111111])

In [140]:
def get_predicted_close_price(_day_index):
        """Predict close price for a given day."""
        open_price = test_data.iloc[_day_index]["Open"]
        (
            predicted_frac_change,
            pred_frac_high,
            pred_frac_low,
        ) = get_most_probable_outcome(_day_index)
        return open_price * (1 + predicted_frac_change)

In [149]:
day_index = 100
get_predicted_close_price(day_index), test_data.iloc[day_index]["Close"]

(4069.108646065848, 4158.240234375)

In [169]:
day_index = 20
(test_data.iloc[day_index]["Open"] - test_data.iloc[day_index]["Close"]), (test_data.iloc[day_index]["Open"] - get_predicted_close_price(day_index))

(-22.98974609375, -9.31916354432451)

In [130]:
day_index=None

In [175]:
def backtest_strategy(
        company = "^GSPC",
        start_date = "2020-01-01",
        end_date = "2023-01-01",
        n_latency_days = 10,
        n_hidden_states = 4,
        days_in_future = 5,

        n_intervals_frac_change=50,
        n_intervals_frac_high=10,
        n_intervals_frac_low=10,
        test_size=0.33
):
        correct = 0
        hmm = GaussianHMM(n_components=n_hidden_states)
        ### Download data
        import yfinance as yfin
        yfin.pdr_override()

        used_data = data.get_data_yahoo(company, start_date, end_date)

        train_data, test_data = train_test_split(
                used_data, test_size=test_size, shuffle=False
                )

        train_data = train_data.drop(["Volume", "Adj Close"], axis=1)
        test_data = test_data.drop(["Volume", "Adj Close"], axis=1)

        frac_change_range = np.linspace(-0.1, 0.1, n_intervals_frac_change)
        frac_high_range = np.linspace(0, 0.1, n_intervals_frac_high)
        frac_low_range = np.linspace(0, 0.1, n_intervals_frac_low)

        _possible_outcomes = np.array(
        list(itertools.product(frac_change_range, frac_high_range, frac_low_range))
        )

        def _extract_features(data):
                """Extract the features - open, close, high, low price - from the Yahooo finance generated dataframe."""
                open_price = np.array(data["Open"])
                close_price = np.array(data["Close"])
                high_price = np.array(data["High"])
                low_price = np.array(data["Low"])

                # We compute the fractional change in high,low and close prices to use as our set of observations
                frac_change = (close_price - open_price) / open_price
                frac_high = (high_price - open_price) / open_price
                frac_low = (open_price - low_price) / open_price

                # Put the observations into one array
                return np.column_stack((frac_change, frac_high, frac_low))
        
        observations = _extract_features(train_data)
        hmm.fit(observations)
        
        def get_most_probable_outcome(_day_index):
                # Use the previous n_latency_days worth of data for predictions
                previous_data_start_index = max(0, _day_index - n_latency_days)
                previous_data_end_index = max(0, _day_index - 1)
                previous_data = test_data.iloc[
                        previous_data_start_index:previous_data_end_index
                ]
                previous_data_features = _extract_features(previous_data)

                outcome_score = []
                # Score all possible outcomes and select the most probable one to use for prediction
                for possible_outcome in _possible_outcomes:
                        total_data = np.row_stack((previous_data_features, possible_outcome))
                        outcome_score.append(hmm.score(total_data))

                # Get the index of the most probable outcome and return it
                most_probable_outcome = _possible_outcomes[np.argmax(outcome_score)]

                return most_probable_outcome
        
        def get_predicted_close_price(_day_index):
                """Predict close price for a given day."""
                open_price = test_data.iloc[_day_index]["Open"]
                (
                predicted_frac_change,
                pred_frac_high,
                pred_frac_low,
                ) = get_most_probable_outcome(_day_index)
                return open_price * (1 + predicted_frac_change)
        
        for day_index in tqdm(range(len(test_data))):
                current_open_price = test_data.iloc[day_index]["Open"]
                current_close_price = test_data.iloc[day_index]["Close"]
                x = current_open_price - current_close_price
                y = current_open_price - get_predicted_close_price(day_index)
                if x*y > 0:
                        correct += 1
        # correct days/all days
        print(correct/len(test_data)) 

backtest_strategy()

[*********************100%%**********************]  1 of 1 completed


100%|██████████| 250/250 [01:19<00:00,  3.15it/s]

0.472


investigate a new model for predicting if the stock will go up or not, rather than the amoutn

In [6]:
import openbb as obb

SyntaxError: invalid syntax (2175187026.py, line 2)

In [2]:
# Get the nominal price returns for 2022
stocks_dataframe = obb.stocks.load(
    "BMWYY",
    start_date="2022-03-01",
    end_date="2023-03-01"
).to_dataframe()
price_start = stocks_dataframe["close"].iloc[0]
price_end = stocks_dataframe["close"].iloc[-1]

nominal_returns = (price_end - price_start) / price_start
print(f"Nominal price returns: {nominal_returns}")

AttributeError: module 'openbb' has no attribute 'stocks'

<bound method ROUTER_stocks.load of /stocks
    /ca
    /fa
    load
    multiples
    news
    /options
    quote
    >